<img src="res/header_cropland.png" alt="title" >

<p ><span style="font-size:14px"><b>Corrector</b> enables early estimation of possible ESG risks based on severe weather events prediction.<br> Key features include:
<br>
- Feature 1
<br>
- Feature 2
<br>
</span></p>

In [1]:
# Necessary library imports
from utils_corrector import *
import ipywidgets as widgets
from IPython.display import display

In [2]:
out = widgets.Output()

def plot_handler(b):    
    min_lat = min_latitude_input.value
    max_lat = max_latitude_input.value
    min_lon = min_longitude_input.value
    max_lon = max_longitude_input.value
    date    = date_slider.value

    if not sanity_check(min_lat, max_lat, min_lon, max_lon):
        return

    file_path = os.path.join(
        "/app/wind/data/03-results",
        f"result_test_{variable_dropdown.value}.csv",
    )
    if os.path.exists(file_path):
        with out:
            plot_pred(file_path, date, min_lat, min_lon, max_lat, max_lon)
    else:
        print(f"File not found: {file_path}")

    

# Pre-check results' existence
assert os.path.exists("/app/wind/data/03-results/result_test_temp.csv"), "no results for temperature found"
assert os.path.exists("/app/wind/data/03-results/result_test_wind.csv"), "no results for wind found"

# Define daterange available
temp_df = load_eval_df('/app/wind/data/03-results/result_test_temp.csv')
wind_df = load_eval_df('/app/wind/data/03-results/result_test_wind.csv')
date_range_temp = temp_df['date']
date_range_wind = wind_df['date']
available_time_range = sorted(list(set(date_range_temp).intersection(set(date_range_wind))))
available_lat_range = sorted(list(set(temp_df['lat']).intersection(set(wind_df['lat']))))
available_lon_range = sorted(list(set(temp_df['lon']).intersection(set(wind_df['lon']))))



# Widgets for latitude and longitude bounds
min_latitude_input = widgets.BoundedFloatText(
    value=min(available_lat_range), min=min(available_lat_range), max=max(available_lat_range), step=0.01, description="Min Latitude:"
)
max_latitude_input = widgets.BoundedFloatText(
    value=max(available_lat_range), min=min(available_lat_range), max=max(available_lat_range), step=0.01, description="Max Latitude:"
)
min_longitude_input = widgets.BoundedFloatText(
    value=min(available_lon_range), min=min(available_lon_range), max=max(available_lon_range), step=0.01, description="Min Longitude:"
)
max_longitude_input = widgets.BoundedFloatText(
    value=max(available_lon_range), min=min(available_lon_range), max=max(available_lon_range), step=0.01, description="Max Longitude:"
)
# Date selector-slider
date_slider = widgets.SelectionSlider(
    options=available_time_range,
    value=available_time_range[0],
    description='Quantile correction date',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
# Variable selection dropdown
variable_dropdown = widgets.Dropdown(
    options=["wind", "temp"], value="wind", description="Variable:"
)

plot_button = widgets.Button(description="Plot Data")
plot_button.on_click(plot_handler)

# Layout configuration
box_layout = widgets.Layout(
    display="flex", flex_flow="column", align_items="center", height="450px"
)
inputs_layout = widgets.HBox(
    [min_latitude_input, max_latitude_input, min_longitude_input, max_longitude_input]
)

vbox = widgets.VBox(
    [
        variable_dropdown,
        date_slider,
        inputs_layout,
        plot_button,
    ],
    layout=box_layout,
)

# Display the UI components
display(vbox)
display(out)

Output()